In [5]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from datetime import datetime
import time

from __future__ import print_function
import json
from pymongo import MongoClient

In [6]:
# wesley
ckey = 'NdQdQm2OALKTJgtCr9mz8SEAK'
csecret= '2GVLXTHcIy3htHqD5NEIX1IGVuTNjiHMHNEXT2OTTlvjP50hKw'
atoken= '980527124133109766-R2NKGIqJjrUvkOALRw7bdzAHRa4FV18'
asecret= 'r5Px3c5gLtXVuNjJI53qoImV1PW6CeXwRyv8gnjqwTzt0'

In [23]:
# 
ckey="m5kFFDfq3b9oh0wKwiteAgd7J"
csecret="tQ2pH1oqhSoy4QQCN25xGZBQ1dMuMtmMUtlSZb9pzO2nNf1S9s"
akey="986390268889923584-u9gAJwMsyZm0fwMeTdU3JgXzsEOWJSm"
asecret="0ZOtMyC4gku2s9LMNx04dsybSaVpnNvQg7lZJh4iuuMCP"

In [9]:
path="../data"
filepath=path+"/"+"alltogether-"+datetime.now().strftime('%Y-%m-%d-%H-%M')+".json"
filepath

'../data/alltogether-2018-05-31-22-32.json'

In [10]:
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open(filepath, 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True


auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, MyListener())
twitterStream.filter(track=["#airpollution","air pollution",# original
                            "forest fire","forest fires",# category 1
                            "water poisoning","water contamination",# category 2 
                            "flood flash","flood water","flood warning" # category 3
                            "earthquake",# category 4
                            "oil spill","pipeline spill","tarsands spill","tankers spill", "fossilfuel spill","petroleum spill",
                           "acid rain","toxic rain","mildew home","mildew infested","mildew basement","mildew removal","mildew growth","mildew inspection","mold flood","mildew flood",
                           "adelgid","aphid","beetle","earwig","insect","locust","louse","moth","psyllid","termites","termite","bites","bugs","bug","bug bites","bugs bites",
                           "allergy","allergins","pollen","dander", "allergy cough","pollen cough","allergins sneeze","pollen sneeze","pollen asthma","dander asthma"
,"dust respiratory","allegy lung"])

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [ ]:
MONGO_HOST= 'mongodb://localhost/twitterdb'  # assuming you have mongoDB installed locally
                                             # and a database called 'twitterdb'
WORDS = ["#airpollution","air pollution",# original
                            "forest fire","forest fires",# category 1
                            "water poisoning","water contamination",# category 2 
                            "flood flash","flood water","flood warning" # category 3
                            "earthquake",# category 4
                            "oil spill","pipeline spill","tarsands spill","tankers spill", "fossilfuel spill","petroleum spill",
                           "acid rain","toxic rain","mildew home","mildew infested","mildew basement","mildew removal","mildew growth","mildew inspection","mold flood","mildew flood",
                           "adelgid","aphid","beetle","earwig","insect","locust","louse","moth","psyllid","termites","termite","bites","bugs","bug","bug bites","bugs bites",
                           "allergy","allergins","pollen","dander", "allergy cough","pollen cough","allergins sneeze","pollen sneeze","pollen asthma","dander asthma"
,"dust respiratory","allegy lung"]
 
 
    class StreamListener(tweepy.StreamListener):    
        #This is a class provided by tweepy to access the Twitter Streaming API. 

        def on_connect(self):
            # Called initially to connect to the Streaming API
            print("You are now connected to the streaming API.")

        def on_error(self, status_code):
            # On error - if an error occurs, display the error / status code
            print('An Error has occured: ' + repr(status_code))
            return False

        def on_data(self, data):
            #This is the meat of the script...it connects to your mongoDB and stores the tweet
            try:
                client = MongoClient(MONGO_HOST)

                # Use twitterdb database. If it doesn't exist, it will be created.
                db = client.twitterdb

                # Decode the JSON from Twitter
                datajson = json.loads(data)

                #grab the 'created_at' data from the Tweet to use for display
                created_at = datajson['created_at']

                #print out a message to the screen that we have collected a tweet
                print("Tweet collected at " + str(created_at))

                #insert the data into the mongoDB into a collection called twitter_search
                #if twitter_search doesn't exist, it will be created.
                db.twitter_search.insert(datajson)
            except Exception as e:
                print(e)

        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        #Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.
        listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
        streamer = tweepy.Stream(auth=auth, listener=listener)
        print("Tracking: " + str(WORDS))
        streamer.filter(track=WORDS)